# 패스트텍스트(FastText)
+ 이번 시간에는 패스트텍스트(FastText)에 대해 배워 보겠습니다.
+ 패스트텍스트(FastText)도 Word Embedding 종류중의 하나로 토큰(단어)를 의미있는 n차원 밀집 Vector 만드는것 입니다.
+ FastText는 페이스북에서 개발하였으며, Word2Vec의 확장판입니다.
+ Word2Vec는 단어를 쪼개질 수 없는 단위로 생각한다면
+ FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주하여 subword를 고려하여 학습합니다.
+ 즉, 단어를 n-gram으로 나누고 나눠진 n-gram의 밀집벡터를 구하고 모두 합산해서 단어 벡터를 만들게 됩니다. 
+ 모르는 단어(Out Of Vocabulary, OOV)에 대해서도 Subword를 통해 유사도가 비슷한 다른 단어를 찾을수 있습니다.
+ 예를 들어 Word2Vec가 'electrofishing' 단어를 모른다면 OOV 에러가 발생하지만 FastText 경우 electro 단어와 fishing 단어를 학습해서 알고 있다면 2개의 단어를 합쳐 electrofishing 단어에 대한 Vector를 만들어 낼수 있습니다.


![fasttext](https://github.com/gzone2000/TEMP_TEST/raw/master/fasttext.PNG)

### 학습목차
##### A. Word2Vec
1. KoNLPy 설치 및 필요한 라이브러리 임포트
2. 훈련 데이터 불러오기 : A방송사 댓글 데이터
3. 훈련 데이터 전처리 : null, 특수문자 제거 및 Okt로 토큰화
4. OKT 이용하여 토콘나이즈
5. Word2Vec 훈련시키고 유사도 측정

#### B. FastText
1. FastText 훈련시키고 유사도 측정

## 실습으로 비교하는 Word2Vec Vs. FastText
- https://wikidocs.net/22883

## A. Word2Vec

### 1. KoNLPy 설치 및 필요한 라이브러리 임포트

In [1]:
# 필요한 라이브러리와 Konlpy 설치후 Okt 임포트
!pip install openpyxl

import pandas as pd
import matplotlib.pyplot as plt
import urllib.request

!python3 -m pip install konlpy
from konlpy.tag import Okt

### 2. 훈련 데이터 불러오기 : A방송사 댓글 데이터

In [3]:
# kbn_train.xlsx, kbn_test.xlsx KBN 댓글 다운로드 및 Pandas read_excel() 함수 활용하여 읽고 합치기

kbn_train = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_train.xlsx', engine='openpyxl')
kbn_test = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_test.xlsx', engine='openpyxl')

train_data = pd.concat([kbn_train, kbn_test])

In [4]:
train_data[:5] # 상위 5개 출력

,Unnamed: 0,data,label
0,0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,2,스토리가 너무 딱딱해서 별로였음,부정
3,3,프로그램A 화이팅하세요!!,긍정
4,4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [5]:
print(len(train_data)) # 리뷰 개수 출력

352


In [6]:
# NULL 값 존재 유무
print(train_data.isnull().sum())

Unnamed: 0    0
data          0
label         0
dtype: int64


### 3. 훈련 데이터 전처리
- null 제거
- 불필요한 특수문자, 숫자등 제거

In [7]:
# NULL 값 존재 유무
print(train_data.isnull().sum())

Unnamed: 0    0
data          0
label         0
dtype: int64


In [8]:
# 'data' 컬럼에 대해 한글 외 문자 제거
train_data['data'] = train_data['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [9]:
train_data[:5] # 상위 5개 출력

,Unnamed: 0,data,label
0,0,재미는 있는데 시간이 짧은게 아쉽네요,긍정
1,1,관련 내용은 우리 직원과는 거리가 멀었음 특히 사내에 홍보할 내용은 아니라고 봄,부정
2,2,스토리가 너무 딱딱해서 별로였음,부정
3,3,프로그램 화이팅하세요,긍정
4,4,높은 곳에 올라가는 모습이 너무 위험해 보여요,부정


In [10]:
train_data[-5:]

,Unnamed: 0,data,label
96,96,작년에 프로그램를 재밋게 봤던 시청자로서 올해의 미니드라마도 매우 기대가 됩니다 올...,긍정
97,97,프로그램 잘 보았습니다 모든일의 바탕은 안전인것 같습니다 모두를 보호하는 최고의 기...,긍정
98,98,위험한 시설에 대한 설명도 부탁드립니다,부정
99,99,구체적으로 어떤 활동을 해왔었고 앞으로 어떤활동을 할건지 잘 설명해줬으면 좋았을 것...,부정
100,100,우리 회사가 잘되면 나라도 잘된다 회사 화이팅 나라 화이팅,긍정


### 4. OKT 이용하여 토콘나이즈

In [11]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [12]:
# 형태소 분석기 OKT를 사용한 토큰화 작업 
# 352개 문장 사용하여 금방 수행함

okt = Okt()
tokenized_data = []
for sentence in train_data['data']:
    temp_X = okt.morphs(sentence, stem=True) # 형태소 분류하여 토큰화, stem=True 어간 추출
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [13]:
print(tokenized_data[0])
print(tokenized_data[1])
print(tokenized_data[-3:])

['재미', '있다', '시간', '짧다', '아쉽다']
['관련', '내용', '우리', '직원', '과는', '거리', '멀다', '특히', '사내', '홍보', '내용', '아니다', '봄']
[['위험하다', '시설', '대한', '설명', '부탁드리다'], ['구체', '적', '어떻다', '활동', '을', '해오다', '앞', '어떤', '활동', '을', '건지다', '자다', '설명', '해주다', '좋다', '것', '같다'], ['우리', '회사', '되다', '나라', '되다', '회사', '화이팅', '나라', '화이팅']]


### 5. Word2Vec 훈련시키고 유사도 측정
- gensim의 Word2Vec을 이용하여 토큰(단어)을 의미있는 Vector로 만듬
- wv(워드투벡터)의 most_similar 이용하여 유사도 측정

In [14]:
# Word2Vec의 하이퍼파라미터값은 다음과 같습니다.
# vector_size = 워드 벡터의 특징 값. 즉, 위드 임베딩 된 벡터의 차원.
# window = 컨텍스트 윈도우 크기
# min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.). 데이터 적어 0으로 지정
# workers = 학습을 위한 프로세스 수
# sg = 0은 CBOW, 1은 Skip-gram.

from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 50, window = 5, min_count = 0, workers = 4, sg = 0)

In [15]:
# Word2Vec로 완성된 임베딩 매트릭스의 크기 확인
# 1240개 단어를 50차원 Vector 표현

model.wv.vectors.shape

(1240, 50)

In [16]:
# 단어 갯수 : 1240
len(model.wv.key_to_index)

1240

In [17]:
# 1240개 단어 나열
model.wv.key_to_index

{'재미': <gensim.models.keyedvectors.Vocab at 0x7f1401f1e7f0>,
 '있다': <gensim.models.keyedvectors.Vocab at 0x7f1401f1e860>,
 '시간': <gensim.models.keyedvectors.Vocab at 0x7f150834edd8>,
 '짧다': <gensim.models.keyedvectors.Vocab at 0x7f150834eb70>,
 '아쉽다': <gensim.models.keyedvectors.Vocab at 0x7f150834e278>,
 '관련': <gensim.models.keyedvectors.Vocab at 0x7f150834e940>,
 '내용': <gensim.models.keyedvectors.Vocab at 0x7f150834ee80>,
 '우리': <gensim.models.keyedvectors.Vocab at 0x7f15082c5828>,
 '직원': <gensim.models.keyedvectors.Vocab at 0x7f15082c54e0>,
 '과는': <gensim.models.keyedvectors.Vocab at 0x7f150905ed68>,
 '거리': <gensim.models.keyedvectors.Vocab at 0x7f150829e7b8>,
 '멀다': <gensim.models.keyedvectors.Vocab at 0x7f150829ea58>,
 '특히': <gensim.models.keyedvectors.Vocab at 0x7f150829eac8>,
 '사내': <gensim.models.keyedvectors.Vocab at 0x7f15082ffb70>,
 '홍보': <gensim.models.keyedvectors.Vocab at 0x7f15082ff748>,
 '아니다': <gensim.models.keyedvectors.Vocab at 0x7f15082ff9b0>,
 '봄': <gensim.models.k

In [18]:
# 우리 단어를 Word2Vec 변환후 Vector 출력 : 우리 단어를 50차원 표현

print(model.wv.get_vector("우리").shape)
print(model.wv.get_vector("우리"))

(50,)
[ 0.00680692  0.00887768  0.01128177  0.00297331 -0.00457122 -0.001619
  0.00296973  0.00146027  0.00606785  0.00857154 -0.00236263  0.00637363
 -0.01134436 -0.00487115  0.01133799 -0.00576806 -0.00795829 -0.00232063
  0.0064191  -0.00494836  0.00507781  0.00644365  0.0148555  -0.01196336
  0.00762938 -0.01321387  0.00884939 -0.01286246  0.00620788 -0.00698379
 -0.00238501  0.00472404  0.00415978  0.00830833  0.01052078 -0.0008736
  0.00266972  0.00941366 -0.00770181  0.01066641  0.01131903 -0.01028315
  0.00911252 -0.00630751 -0.0082988  -0.01319006  0.00748939  0.00771675
  0.00964762 -0.00389842]


In [19]:
# 우리 키워드 유사도 조회
print(model.wv.most_similar("우리"))

[('자다', 0.5490800738334656), ('너무', 0.5254648923873901), ('결론', 0.4955175817012787), ('앞', 0.4812023639678955), ('에서도', 0.4757255017757416), ('내용', 0.4702262282371521), ('것', 0.46932098269462585), ('부분', 0.456369549036026), ('대한', 0.4500693678855896), ('을', 0.44577986001968384)]


In [20]:
# 나라 키워드 유사도 조회
print(model.wv.most_similar("나라"))

[('생각', 0.4233657419681549), ('구매', 0.4164330065250397), ('떠오르다', 0.38794776797294617), ('악', 0.3826710283756256), ('일어서다', 0.3779218792915344), ('훈훈하다', 0.37621158361434937), ('피싱', 0.36631450057029724), ('중심', 0.3625722825527191), ('이어지다', 0.362432599067688), ('기르다', 0.3535640239715576)]


In [21]:
# 우리나라 키워드 유사도 조회 : 단어 집합(Vocabulary)에 존재하지 않는 단어 입력으로 OOV 에러 발생
print(model.wv.most_similar("우리나라"))

KeyError: "word '우리나라' not in vocabulary"

## B. FastText

### 1. FastText 훈련시키고 유사도 측정
- gensim의 FastText을 이용하여 토큰(단어)을 의미있는 Vector로 만듬
- FastText wv(워드투벡터)의 most_similar 이용하여 유사도 측정

In [22]:
# FastText 이용하여 토큰(단어)를 의미있는 Vector 만들기

# FastText의 하이퍼파라미터값은 다음과 같습니다.
# vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
# window = 컨텍스트 윈도우 크기
# min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.). 데이터 적어 0으로 지정
# workers = 학습을 위한 프로세스 수
# sg = 0은 CBOW, 1은 Skip-gram.

from gensim.models import FastText
FT_model = FastText(sentences = tokenized_data, vector_size=50, window=5, min_count=0, workers=4, sg=0)

In [23]:
# FastText로 완성된 임베딩 매트릭스의 크기 확인
# 1240개 단어를 50차원 Vector 표현

FT_model.wv.vectors.shape

(1240, 50)

In [24]:
# 1240개 단어 나열
FT_model.wv.key_to_index

{'재미': <gensim.models.keyedvectors.Vocab at 0x7f14cd108dd8>,
 '있다': <gensim.models.keyedvectors.Vocab at 0x7f14cd108e10>,
 '시간': <gensim.models.keyedvectors.Vocab at 0x7f14cd108e48>,
 '짧다': <gensim.models.keyedvectors.Vocab at 0x7f14cd108e80>,
 '아쉽다': <gensim.models.keyedvectors.Vocab at 0x7f14cd108eb8>,
 '관련': <gensim.models.keyedvectors.Vocab at 0x7f14cd108ef0>,
 '내용': <gensim.models.keyedvectors.Vocab at 0x7f14cd108f28>,
 '우리': <gensim.models.keyedvectors.Vocab at 0x7f14cd108f60>,
 '직원': <gensim.models.keyedvectors.Vocab at 0x7f14cd108f98>,
 '과는': <gensim.models.keyedvectors.Vocab at 0x7f14cd108fd0>,
 '거리': <gensim.models.keyedvectors.Vocab at 0x7f14c4177048>,
 '멀다': <gensim.models.keyedvectors.Vocab at 0x7f14c4177080>,
 '특히': <gensim.models.keyedvectors.Vocab at 0x7f14c41770b8>,
 '사내': <gensim.models.keyedvectors.Vocab at 0x7f14c41770f0>,
 '홍보': <gensim.models.keyedvectors.Vocab at 0x7f14c4177128>,
 '아니다': <gensim.models.keyedvectors.Vocab at 0x7f14c4177160>,
 '봄': <gensim.models.k

In [25]:
# 우리 키워드 유사도 조회
print(FT_model.wv.most_similar("우리"))

[('기대하다', 0.6433944702148438), ('안전하다', 0.5887050032615662), ('상당하다', 0.582152247428894), ('탄탄하다', 0.5637544989585876), ('연결하다', 0.5579863786697388), ('간략하다', 0.5440536141395569), ('좋아하다', 0.5388294458389282), ('유익하다', 0.5384096503257751), ('산만하다', 0.5245428085327148), ('어색하다', 0.5190675258636475)]


In [26]:
# 나라 키워드 유사도 조회
print(FT_model.wv.most_similar("나라"))

[('앵글', 0.4887971580028534), ('미니', 0.478254497051239), ('친구', 0.44861721992492676), ('기원', 0.42291373014450073), ('맞다', 0.4066935181617737), ('지난', 0.39479124546051025), ('보이지', 0.38855764269828796), ('고려', 0.36886051297187805), ('트렌디', 0.3558017909526825), ('돕다', 0.340631365776062)]


In [27]:
# 단어 집합(Vocabulary)에 존재하지 않는 '우리나라' 단어 입력해도 OOV 에러 발생하지 않음
# FastText에서 '우리나라' 단어가 존재하지 않지만 다른 단어 '우리'와 '나라'라는 내부 단어가 있었다면, FastText는 '우리나라' 단어의 벡터를 얻을 수 있습니다.

print(FT_model.wv.most_similar("우리나라"))

[('우리', 0.42920324206352234), ('시점', 0.4107832908630371), ('공', 0.40139687061309814), ('담기다', 0.39992570877075195), ('패인', 0.3973265588283539), ('분들', 0.3832615911960602), ('개', 0.3746064603328705), ('알아차리다', 0.37041792273521423), ('것', 0.36530956625938416), ('나라', 0.35904163122177124)]


In [28]:
# 우리, 나라, 우리나라 단어에 대한 Vector 출력

print(FT_model.wv.get_vector('우리'))
print(FT_model.wv.get_vector('나라'))
print(FT_model.wv.get_vector('우리나라'))

[-0.00848174  0.00481808  0.00579075  0.01328798 -0.0043528   0.00763287
  0.00090899 -0.00013889  0.00175994  0.00397417  0.00634739  0.00758098
 -0.00101744 -0.00241409  0.00266464 -0.0096597   0.01390492 -0.00648851
 -0.00517867  0.01261204  0.00804102  0.00078501 -0.01412871  0.00472138
  0.00390507 -0.01222963 -0.00073154 -0.01301737 -0.00159045  0.00101591
  0.01408675 -0.00070042 -0.00196168 -0.01045747 -0.00421656 -0.00312112
  0.01189917 -0.0118549   0.00041495 -0.00631899 -0.00478612  0.01124396
 -0.00168772 -0.00184591  0.00377715 -0.00555186  0.00451501  0.00098782
  0.00262078 -0.00655085]
[ 1.3667040e-03  5.5322616e-04  1.3400546e-02  9.6126267e-04
  5.3879677e-04  5.9217489e-03  3.5811798e-03  6.7434227e-04
  1.3955590e-02  2.8886332e-04 -2.4968206e-03 -1.5571951e-03
 -5.7065533e-04  2.3544140e-04 -8.2419906e-04  7.6775160e-03
 -1.2364856e-03  2.7141273e-03  5.2598040e-03 -5.7516564e-03
 -6.7003164e-03  1.6708742e-04  4.2176601e-03 -3.1661389e-03
  2.7772170e-03  7.17990

# 배운 내용 정리
1. Word Embedding : 단어를 의미있는 밀집 벡터로 표현하기
2. Word Embedding 방법에는 Word2Vec, FastText 등 있습니다.
3. 이전 실습에서 배운 Word2Vec를 다시 실습해 보고 '우리나라' 같은 모르는 단어에 대해서 OOV 에러 발생한다는것 알수 있었으며
4. FastText 경우 똑같이 '우리나라' 단어를 모르지만, Subword를 통해 모르는 단어에 대해서도 벡터를 만들어 낼수 있었습니다.
5. Word2Vec와 FastText간의 차이를 알수 있었으며, 둘다 Word Embedding 이라는점을 기억하면 되겠습니다.